
# Projeto nº 4 - Aprendizagem Automática
### Introdução à Inteligência Artificial edição 2020/21


## Grupo: 22

### Elementos do Grupo

Nome: André Firmino

Número: 44999

Nome: Joao Janeiro

Número: 52779

Nome: Nuno Estalagem

Número 52828

### Imports
De modo a conseguir concretizar este projeto,

In [12]:
from utilsAA import *
import numpy as np
from random import*
import sklearn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree # árvore de decisão
from sklearn.neighbors import KNeighborsClassifier # k-NN
from sklearn.model_selection import train_test_split, cross_val_score

### Etapa 1 - *Load* de Dados
Nesta fase do nosso projeto, temos como intenção o *load* dos 
dados. Esta fase é muito importante, já que, a partir dos dados dele,
vai ser gerada, numa fase posterior, uma árvore de decisão, que nos auxiliará na extrapolação e previsão
do melhor modelo possível, a ser aplicado no 'test.csv'. Como sugerido pelo enunciado, e tendo em vista o *load* de dados, recorremos à função *load_data* do módulo utilsAA.


In [13]:
data,target,feature_names,target_names=load_data('airline.csv')


### Etapa 2 - Processamento dos Dados
Nesta fase do nosso projeto, verificámos se os dados de que fizemos *load* estão prontos a serem usados pelos algoritmos. 
Para que tal seja possível, temos de ter em atenção que os atributos do *airline.csv* são categóricos e, deste modo, necessitam de ser codificados, através do método *encode_feature*. Uma vez que o client_id é irrelevante, podemos remover a primeira coluna dos dados e do vector com o nome dos atributos. Finalmente, recorremos à função one_hot_encode_feature, que recebe os dados completos (*data*), o número da coluna que se pretende codificar(4) e o vector com o nome dos atributos (*feature_names*), tem como resultado, apagar essa coluna e adicionar no fim dos dados o número de colunas correspondentes ao número de categorias existentes nesse atributo categórico. A partir deste momento, todos os atributos são numéricos e podemos criar um modelo com árvores de decisão.

In [14]:
feature_names = feature_names[1:]
data = data[:,1:]
data[:,0]=encode_feature(data[:,0])
data[:,1]=encode_feature(data[:,1])
data[:,3]=encode_feature(data[:,3])
data, feature_names = one_hot_encode_feature(data, 4, feature_names)


### Etapa 3 - Procura do Melhor Modelo
Na fase 3, experimentámos os algoritmos das árvores de decisão e k vizinhos mais próximos, variando os respetivos parâmetros. A escolha do melhor modelo a aplicar no conjunto de teste fornecido foi feita, tendo em conta um algoritmo de *splitting*, para cada tipo de algoritmo (árvores de decisão e k vizinhos mais próximos). No contexto da aprendizagem automática, o *splitting* serve para aferir a qualidade do modelo de aprendizagem, dividindo os dados que temos em duas "classes": uma de treino e outra de teste. Uma das partes (treino) é usada para desenvolver um modelo preditivo, enquanto que outra (teste) é usada para avaliar desempenho do algoritmo. No código abaixo, está presente a forma de implementação dos algoritmos referidos acima, sendo que a ideia chave é calcular/averiguar quais são os melhores parâmetros a usar na árvore de decisão ou no k vizinhos mais próximos, conforme o que pretendermos.

In [15]:
def best_splitter_dec_tree():
    max_maior=0
    value=0
    value2=0
    for x in range(21):
        for y in range(21):
            dtc = DecisionTreeClassifier(criterion='entropy',min_samples_split=x, max_depth=y)
            scores = cross_val_score(dtc,
                             X=data,
                             y=target,
                             cv=10,
                             n_jobs=-1
                            )
            if np.mean(scores)>max_maior:
                max_maior=np.mean(scores)
                value=x
                value2=y
    print("A maior percentagem eh " + str(max_maior)+ " e utilizamos valor: " + str(value) + " " +str(value2));
    return(value,value2)

valores=best_splitter_dec_tree()
dtc = DecisionTreeClassifier(criterion='entropy',min_samples_split=valores[0],max_depth=valores[1])
scores = cross_val_score(dtc,
                         X=data,
                         y=target,
                         cv=10,
                         n_jobs=-1
                        )
print('CV accuracy:', *scores, sep='\n\t')
print('Average CV accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))





def best_splitter_neighbors():
    max_maior=0
    neighbors=0
    for x in range(31):
        dtc = KNeighborsClassifier(n_neighbors=x)
        scores = cross_val_score(dtc,
                             X=data,
                             y=target,
                             cv=10,
                             n_jobs=-1
                            )
        if np.mean(scores)>max_maior:
            max_maior=np.mean(scores)
            neighbors=x
    print("A maior percentagem eh " + str(max_maior)+ " e utilizamos valor: " +str(neighbors));
    return neighbors
neighbors=best_splitter_neighbors()
ctc =  KNeighborsClassifier(n_neighbors=neighbors)
scores = cross_val_score(ctc,
                         X=data,
                         y=target,
                         cv=10,
                         n_jobs=-1
                        )
print('CV accuracy:', *scores, sep='\n\t')
print('Average CV accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

A maior percentagem eh 0.9061771561771561 e utilizamos valor: 13 6
CV accuracy:
	0.8205128205128205
	0.9487179487179487
	0.9230769230769231
	0.9358974358974359
	0.9102564102564102
	0.9102564102564102
	0.9102564102564102
	0.8717948717948718
	0.8571428571428571
	0.961038961038961
Average CV accuracy: 0.905 +/- 0.041
A maior percentagem eh 0.6774225774225775 e utilizamos valor: 30
CV accuracy:
	0.6538461538461539
	0.6282051282051282
	0.6666666666666666
	0.7051282051282052
	0.6794871794871795
	0.7564102564102564
	0.6282051282051282
	0.6666666666666666
	0.6753246753246753
	0.7142857142857143
Average CV accuracy: 0.677 +/- 0.037


### Etapa 4 - Gravação das Previsões
Após a escolha do melhor modelo,calculado na alínea anterior, fizemos load dos dados de teste com a mesma função *load_data*, mas usando o argumento *testdata=True*, o que nos permite ler os dados sem a coluna das classes (que não irá existir neste conjunto de dados). De seguida, fazemos *fit* à árvore que é calculada na alínea número 3, gerando uma árvore de decisão. Para esta árvore, são chamados os valores de *test.csv*, prevendo, demonstrado na linha que diz *var = dtc.predict(data_test)*. E tendo em conta a árvore que fizemos a partir dos dados na 3º passo, verificamos se se consegue prever os resultados do *teste.csv*. Depois de fazer as previsões com o modelo escolhido, usámos a função *save_data* para gravar as nossas previsões para o conjunto de teste, gerando um ficheiro CSV com o nome *IIA2021-proj4-22.csv*.

In [16]:
data_test, features_names_test = load_data("test.csv",testdata=True)

In [17]:
features_names_test =  features_names_test[1:]
data_test = data_test[:,1:]
data_test[:,0]=encode_feature(data_test[:,0])
data_test[:,1]=encode_feature(data_test[:,1])
data_test[:,3]=encode_feature(data_test[:,3])
data_test, features_names_test = one_hot_encode_feature(data_test, 4, features_names_test)
dtc = DecisionTreeClassifier(criterion='entropy',min_samples_split=valores[0],max_depth=valores[1])
dtc.fit(data,target)
var=dtc.predict(data_test)
"""
plt.figure(figsize=[14,10])
plot_tree(dtc,
          feature_names=feature_names,
          class_names=target,
          filled=True, rounded=True)
plt.show()
"""
save_data('IIA2021-proj4-22.csv',var)
